In [211]:
import pandas as pd
import numpy as np
from selenium import webdriver
from bs4 import BeautifulSoup
import time
selenium_location='/Users/thomas/Downloads/phantomjs-2.1.1-macosx/bin/phantomjs'
chrome_location='/Users/thomas/Downloads/chromedriver'

In [ ]:
cases=[]
text=[]

In [305]:
safe_sleep=6.
download_sleep=15.
base='https://www.oyez.org/'
for i in range(2009,2014):
    driver1=webdriver.chrome.webdriver.WebDriver(executable_path=chrome_location)
    driver1.get(base+'cases/'+str(i))
    time.sleep(safe_sleep)
    html=driver1.page_source
    soup=BeautifulSoup(html,'html')
    driver1.quit()
    for case_name,case_url in [[x.a.text,x.a['href']] for x in soup.findAll('h2')]:
        if [case_name,case_url] not in cases:
            cases.append([case_name,case_url])
            driver2=webdriver.chrome.webdriver.WebDriver(executable_path=chrome_location)
            driver2.get(base+case_url)
            time.sleep(safe_sleep)
            html=driver2.page_source
            soup2=BeautifulSoup(html,'html')
            driver2.quit()
            for url in [x['iframe-url'] for x in soup2.findAll('a',{'ng-click':'openModal()'}) if 'apps.oyez' in x['iframe-url']]:
                driver3=webdriver.chrome.webdriver.WebDriver(executable_path=chrome_location)
                driver3.get(url)
                time.sleep(safe_sleep)
                html=driver3.page_source
                soup3=BeautifulSoup(html,'html')
                if len(soup3.findAll('p',{'ng-repeat':'textBlock in turn.text_blocks'}))<1:
                    time.sleep(3)
                for chunk in soup3.findAll('p',{'ng-repeat':'textBlock in turn.text_blocks'}):
                    text.append([case_url,chunk['start-time'],chunk['stop-time'],chunk.text])
                try:
                    element=driver3.find_element_by_xpath('//div[@class="control-wrapper share"]/button[1]')
                    element.click()
                except:
                    time.sleep(safe_sleep)
                    element=driver3.find_element_by_xpath('//div[@class="control-wrapper share"]/button[1]')
                    element.click()
                time.sleep(1.)
                try:
                    element=driver3.find_element_by_xpath('//button[@class="download"]')
                    element.click()
                except:
                    time.sleep(safe_sleep)
                    element=driver3.find_element_by_xpath('//button[@class="download"]')
                    element.click()
                time.sleep(download_sleep)
                driver3.close()
        

pd.DataFrame(cases,columns=['name','number']).to_csv('case_information',index=False)

pd.DataFrame([[x[0],x[1],x[2],''.join([unicode(y) for y in x[3] if ord(y)<128])] for x in text],columns=['case','start_time','stop_time','dialogue']).to_csv('transcripts',index=False)

KeyboardInterrupt: 

In [306]:
pd.DataFrame(cases,columns=['name','number']).to_csv('case_information',index=False)

pd.DataFrame([[x[0],x[1],x[2],''.join([unicode(y) for y in x[3] if ord(y)<128])] for x in text],columns=['case','start_time','stop_time','dialogue']).to_csv('transcripts',index=False)

In [307]:
len(text)

203618